In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os

#### Load GloVe embeddings

In [2]:
embeddings_index = {}

f = open ('glove.6B.100d.txt','r', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:])
    embeddings_index[word]=coefs

f.close()
print ('Found %s word embeddings'%(len(embeddings_index)))

Found 400000 word embeddings


### Load comments

In [3]:
input_texts = []
target_texts = []

for line in open('./data/poetry/robert_frost.txt'):
    line = line.rstrip()
    input_line = '<sos>'+line
    target_line = line+'<eos>'
    input_texts.append(input_line)
    target_texts.append(target_line)
    
all_lines = input_texts + target_texts

In [4]:
MAX_WORDS=20000
BATCH_SIZE=128
EPOCHS=2000
OOV_TOKEN=0
EMBEDDING_DIM = 100
MAX_SEQ_LENGTH=100
VALIDATION_SPLIT_RATIO= 0.2
LSTM_UNITS=64

### Tokenize comments

In [5]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=MAX_WORDS, oov_token=OOV_TOKEN, filters='')
tokenizer.fit_on_texts(all_lines)
word2Idx = tokenizer.word_index

print ('Found %d unique words'%(len(word2Idx)))
print (word2Idx['girl'])

input_sequences = tokenizer.texts_to_sequences(input_texts)
target_sequences = tokenizer.texts_to_sequences(target_texts)
print ('Found %d input sequences'%(len(input_sequences)))
print ('Found %d output sequences'%(len(target_sequences)))

max_seq_len_from_data = min (MAX_SEQ_LENGTH, max(len(s) for s in input_sequences))
print ('max seq length is %d'%(max_seq_len_from_data))

padded_input_sequences = tf.keras.preprocessing.sequence.pad_sequences(input_sequences, padding='post', maxlen=max_seq_len_from_data)
padded_target_sequences = tf.keras.preprocessing.sequence.pad_sequences(target_sequences, padding='post', maxlen=max_seq_len_from_data)

print ('Created %d padded input sequences'%(len(padded_input_sequences)))
print ('Created %d padded target sequences'%(len(padded_target_sequences)))

assert ('<sos>' in word2Idx)
assert ('<eos>' in word2Idx)

Found 4617 unique words
1268
Found 1581 input sequences
Found 1581 output sequences
max seq length is 11
Created 1581 padded input sequences
Created 1581 padded target sequences


In [6]:
num_words = min (MAX_WORDS, len(word2Idx)+1)
print ('Min words to be considered are %d'%(num_words))

loaded_embeddings_matrix = np.zeros((num_words, EMBEDDING_DIM))

for word, i in word2Idx.items():
    if (i<num_words):
        if word in embeddings_index.keys():
            embedding_vector = embeddings_index[word]
            loaded_embeddings_matrix[i] = embedding_vector

print (loaded_embeddings_matrix.shape)

Min words to be considered are 4618
(4618, 100)


In [7]:
for i, seq in enumerate(padded_target_sequences):
    print (seq)
    if (i>2):
        break

[ 136  574  575    8    4  576 3470    0    0    0    0]
[   5  577    6   69   31  987 3471    0    0    0    0]
[   5   27   28  988  166    6 3472    0    0    0    0]
[   5  184   72   28   17  140   17    6 3473    0    0]


### Important Step:

If we look at this problem statement, we have a set of input sequences and corresponding target sequences.

Input Seq (T*D)-> Target Seq (T'*D')

    T -> number of input sequences
    D -> length of each input sequence

    T' -> number of target sequences
    D' -> length of each target sequence.

In a seq2seq scenario the target value needs to be one-hot encoded and that's what we are doing below.

In [8]:
one_hot_targets = np.zeros((len(target_sequences), max_seq_len_from_data, num_words))
print (one_hot_targets.shape)
for i, seq in enumerate(padded_target_sequences):
    for j, word in enumerate(seq):
        if (word>0):
            one_hot_targets[i,j,word]=1

(1581, 11, 4618)


### Build Model

In [9]:
embed_layer = tf.keras.layers.Embedding(input_dim = num_words,
                               output_dim = EMBEDDING_DIM,
                               input_length=max_seq_len_from_data, 
                               embeddings_initializer=tf.keras.initializers.Constant(loaded_embeddings_matrix),
                               trainable=True)

In [10]:
input_ = tf.keras.layers.Input(shape=(max_seq_len_from_data))
initial_h = tf.keras.layers.Input(shape=(LSTM_UNITS,))
initial_c = tf.keras.layers.Input(shape=(LSTM_UNITS,))

x = embed_layer(input_)

lstm_layer_0 = tf.keras.layers.LSTM(units=LSTM_UNITS, return_sequences=True, return_state=True)
x,_,_ = lstm_layer_0(x)

#stm_layer_1 = tf.keras.layers.LSTM(units=LSTM_UNITS, return_sequences=True, return_state=True)
#x,_,_ = lstm_layer_1(x)

dense_layer = tf.keras.layers.Dense(num_words, activation='softmax')
output = dense_layer(x)

model = tf.keras.models.Model([input_, initial_h, initial_c], output)

model.compile (optimizer=tf.keras.optimizers.Adam(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

W0913 00:21:11.835248  9100 deprecation.py:506] From C:\MachineLearning\anaconda\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 11)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 11, 100)      461800      input_1[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 11, 64), (No 42240       embedding[0][0]                  
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 64)]         0                                            
______________________________________________________________________________________________

In [11]:
from sklearn.model_selection import train_test_split

train_sentences, test_sentences, train_targets, test_targets = train_test_split (padded_input_sequences, one_hot_targets)

In [12]:
h = np.zeros((len(train_sentences),LSTM_UNITS))
c = np.zeros((len(train_sentences),LSTM_UNITS))
model.fit ([train_sentences,h,c], train_targets, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=VALIDATION_SPLIT_RATIO)

W0913 00:21:13.312250  9100 deprecation.py:323] From C:\MachineLearning\anaconda\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 948 samples, validate on 237 samples
Epoch 1/2000
948/948 [==============================] - 4s 4ms/sample - loss: 4.9743 - acc: 0.0519 - val_loss: 4.5060 - val_acc: 0.0399
Epoch 2/2000
948/948 [==============================] - 3s 4ms/sample - loss: 4.1644 - acc: 0.0384 - val_loss: 4.1600 - val_acc: 0.0318
Epoch 3/2000
948/948 [==============================] - 3s 4ms/sample - loss: 3.8241 - acc: 0.0399 - val_loss: 3.9862 - val_acc: 0.0564
Epoch 4/2000
948/948 [==============================] - 4s 4ms/sample - loss: 3.5196 - acc: 0.0690 - val_loss: 3.8845 - val_acc: 0.0740
Epoch 5/2000
948/948 [==============================] - 3s 4ms/sample - loss: 3.2458 - acc: 0.0953 - val_loss: 3.7087 - val_acc: 0.1036
Epoch 6/2000
948/948 [==============================] - 3s 4ms/sample - loss: 2.9923 - acc: 0.1299 - val_loss: 3.5634 - val_acc: 0.1461
Epoch 7/2000
948/948 [==============================] - 3s 4ms/sample - loss: 2.7394 - acc: 0.1807 - val_loss: 3.4165 - val_acc: 0.1895
Ep

948/948 [==============================] - 4s 4ms/sample - loss: 0.0197 - acc: 0.6266 - val_loss: 2.5394 - val_acc: 0.4139
Epoch 61/2000
948/948 [==============================] - 3s 4ms/sample - loss: 0.0194 - acc: 0.6263 - val_loss: 2.5434 - val_acc: 0.4135
Epoch 62/2000
948/948 [==============================] - 3s 4ms/sample - loss: 0.0187 - acc: 0.6267 - val_loss: 2.5429 - val_acc: 0.4131
Epoch 63/2000
948/948 [==============================] - 3s 4ms/sample - loss: 0.0181 - acc: 0.6269 - val_loss: 2.5418 - val_acc: 0.4139
Epoch 64/2000
948/948 [==============================] - 3s 4ms/sample - loss: 0.0174 - acc: 0.6264 - val_loss: 2.5400 - val_acc: 0.4124
Epoch 65/2000
948/948 [==============================] - 3s 4ms/sample - loss: 0.0171 - acc: 0.6270 - val_loss: 2.5429 - val_acc: 0.4124
Epoch 66/2000
948/948 [==============================] - 3s 4ms/sample - loss: 0.0163 - acc: 0.6272 - val_loss: 2.5431 - val_acc: 0.4135
Epoch 67/2000
948/948 [==============================] 

Epoch 120/2000
948/948 [==============================] - 3s 4ms/sample - loss: 0.0045 - acc: 0.6289 - val_loss: 2.5586 - val_acc: 0.4120
Epoch 121/2000
948/948 [==============================] - 3s 4ms/sample - loss: 0.0045 - acc: 0.6290 - val_loss: 2.5581 - val_acc: 0.4124
Epoch 122/2000
948/948 [==============================] - 3s 4ms/sample - loss: 0.0044 - acc: 0.6288 - val_loss: 2.5593 - val_acc: 0.4124
Epoch 123/2000
948/948 [==============================] - 3s 4ms/sample - loss: 0.0043 - acc: 0.6291 - val_loss: 2.5600 - val_acc: 0.4120
Epoch 124/2000
948/948 [==============================] - 3s 4ms/sample - loss: 0.0043 - acc: 0.6289 - val_loss: 2.5600 - val_acc: 0.4120
Epoch 125/2000
948/948 [==============================] - 3s 4ms/sample - loss: 0.0042 - acc: 0.6291 - val_loss: 2.5603 - val_acc: 0.4120
Epoch 126/2000
948/948 [==============================] - 3s 4ms/sample - loss: 0.0044 - acc: 0.6290 - val_loss: 2.5619 - val_acc: 0.4120
Epoch 127/2000
948/948 [==========

948/948 [==============================] - 3s 4ms/sample - loss: 0.0019 - acc: 0.6293 - val_loss: 2.5769 - val_acc: 0.4116
Epoch 180/2000
948/948 [==============================] - 3s 4ms/sample - loss: 0.0019 - acc: 0.6293 - val_loss: 2.5769 - val_acc: 0.4116
Epoch 181/2000
948/948 [==============================] - 3s 4ms/sample - loss: 0.0019 - acc: 0.6293 - val_loss: 2.5774 - val_acc: 0.4112
Epoch 182/2000
948/948 [==============================] - 3s 4ms/sample - loss: 0.0019 - acc: 0.6293 - val_loss: 2.5783 - val_acc: 0.4108
Epoch 183/2000
948/948 [==============================] - 3s 4ms/sample - loss: 0.0019 - acc: 0.6293 - val_loss: 2.5788 - val_acc: 0.4112
Epoch 184/2000
948/948 [==============================] - 3s 4ms/sample - loss: 0.0019 - acc: 0.6293 - val_loss: 2.5795 - val_acc: 0.4112
Epoch 185/2000
948/948 [==============================] - 3s 4ms/sample - loss: 0.0018 - acc: 0.6293 - val_loss: 2.5790 - val_acc: 0.4112
Epoch 186/2000
948/948 [=========================

...

948/948 [==============================] - 3s 4ms/sample - loss: 1.6832e-06 - acc: 0.6293 - val_loss: 2.7798 - val_acc: 0.4127
Epoch 1734/2000
948/948 [==============================] - 3s 4ms/sample - loss: 1.6784e-06 - acc: 0.6293 - val_loss: 2.7799 - val_acc: 0.4127
Epoch 1735/2000
948/948 [==============================] - 3s 4ms/sample - loss: 1.6737e-06 - acc: 0.6293 - val_loss: 2.7801 - val_acc: 0.4127
Epoch 1736/2000
948/948 [==============================] - 3s 4ms/sample - loss: 1.6680e-06 - acc: 0.6293 - val_loss: 2.7801 - val_acc: 0.4127
Epoch 1737/2000
948/948 [==============================] - 3s 4ms/sample - loss: 1.6633e-06 - acc: 0.6293 - val_loss: 2.7803 - val_acc: 0.4127
Epoch 1738/2000
948/948 [==============================] - 3s 4ms/sample - loss: 1.6584e-06 - acc: 0.6293 - val_loss: 2.7804 - val_acc: 0.4127
Epoch 1739/2000
948/948 [==============================] - 3s 4ms/sample - loss: 1.6536e-06 - acc: 0.6293 - val_loss: 2.7805 - val_acc: 0.4127
Epoch 1740/2000

948/948 [==============================] - 3s 4ms/sample - loss: 1.4375e-06 - acc: 0.6293 - val_loss: 2.7839 - val_acc: 0.4127
Epoch 1791/2000
948/948 [==============================] - 3s 4ms/sample - loss: 1.4337e-06 - acc: 0.6293 - val_loss: 2.7839 - val_acc: 0.4127
Epoch 1792/2000
948/948 [==============================] - 3s 4ms/sample - loss: 1.4293e-06 - acc: 0.6293 - val_loss: 2.7840 - val_acc: 0.4127
Epoch 1793/2000
948/948 [==============================] - 3s 4ms/sample - loss: 1.4258e-06 - acc: 0.6293 - val_loss: 2.7841 - val_acc: 0.4127
Epoch 1794/2000
948/948 [==============================] - 3s 4ms/sample - loss: 1.4214e-06 - acc: 0.6293 - val_loss: 2.7841 - val_acc: 0.4127
Epoch 1795/2000
948/948 [==============================] - 3s 4ms/sample - loss: 1.4178e-06 - acc: 0.6293 - val_loss: 2.7842 - val_acc: 0.4127
Epoch 1796/2000
948/948 [==============================] - 3s 4ms/sample - loss: 1.4139e-06 - acc: 0.6293 - val_loss: 2.7842 - val_acc: 0.4127
Epoch 1797/2000

948/948 [==============================] - 3s 4ms/sample - loss: 1.2358e-06 - acc: 0.6293 - val_loss: 2.7874 - val_acc: 0.4131
Epoch 1848/2000
948/948 [==============================] - 3s 4ms/sample - loss: 1.2323e-06 - acc: 0.6293 - val_loss: 2.7874 - val_acc: 0.4131
Epoch 1849/2000
948/948 [==============================] - 3s 4ms/sample - loss: 1.2295e-06 - acc: 0.6293 - val_loss: 2.7876 - val_acc: 0.4131
Epoch 1850/2000
948/948 [==============================] - 3s 4ms/sample - loss: 1.2263e-06 - acc: 0.6293 - val_loss: 2.7878 - val_acc: 0.4131
Epoch 1851/2000
948/948 [==============================] - 3s 4ms/sample - loss: 1.2237e-06 - acc: 0.6293 - val_loss: 2.7876 - val_acc: 0.4131
Epoch 1852/2000
948/948 [==============================] - 3s 4ms/sample - loss: 1.2205e-06 - acc: 0.6293 - val_loss: 2.7877 - val_acc: 0.4131
Epoch 1853/2000
948/948 [==============================] - 3s 4ms/sample - loss: 1.2176e-06 - acc: 0.6293 - val_loss: 2.7878 - val_acc: 0.4131
Epoch 1854/2000

948/948 [==============================] - 3s 4ms/sample - loss: 1.0735e-06 - acc: 0.6293 - val_loss: 2.7908 - val_acc: 0.4131
Epoch 1905/2000
948/948 [==============================] - 3s 4ms/sample - loss: 1.0705e-06 - acc: 0.6293 - val_loss: 2.7908 - val_acc: 0.4131
Epoch 1906/2000
948/948 [==============================] - 3s 4ms/sample - loss: 1.0681e-06 - acc: 0.6293 - val_loss: 2.7909 - val_acc: 0.4131
Epoch 1907/2000
948/948 [==============================] - 3s 4ms/sample - loss: 1.0654e-06 - acc: 0.6293 - val_loss: 2.7910 - val_acc: 0.4131
Epoch 1908/2000
948/948 [==============================] - 3s 4ms/sample - loss: 1.0628e-06 - acc: 0.6293 - val_loss: 2.7910 - val_acc: 0.4131
Epoch 1909/2000
948/948 [==============================] - 3s 4ms/sample - loss: 1.0608e-06 - acc: 0.6293 - val_loss: 2.7910 - val_acc: 0.4131
Epoch 1910/2000
948/948 [==============================] - 3s 4ms/sample - loss: 1.0580e-06 - acc: 0.6293 - val_loss: 2.7911 - val_acc: 0.4131
Epoch 1911/2000

948/948 [==============================] - 3s 4ms/sample - loss: 9.3773e-07 - acc: 0.6293 - val_loss: 2.7938 - val_acc: 0.4131
Epoch 1962/2000
948/948 [==============================] - 3s 4ms/sample - loss: 9.3524e-07 - acc: 0.6293 - val_loss: 2.7938 - val_acc: 0.4131
Epoch 1963/2000
948/948 [==============================] - 3s 4ms/sample - loss: 9.3313e-07 - acc: 0.6293 - val_loss: 2.7939 - val_acc: 0.4131
Epoch 1964/2000
948/948 [==============================] - 3s 4ms/sample - loss: 9.3126e-07 - acc: 0.6293 - val_loss: 2.7939 - val_acc: 0.4131
Epoch 1965/2000
948/948 [==============================] - 3s 4ms/sample - loss: 9.2854e-07 - acc: 0.6293 - val_loss: 2.7939 - val_acc: 0.4131
Epoch 1966/2000
948/948 [==============================] - 3s 4ms/sample - loss: 9.2667e-07 - acc: 0.6293 - val_loss: 2.7939 - val_acc: 0.4131
Epoch 1967/2000
948/948 [==============================] - 3s 4ms/sample - loss: 9.2455e-07 - acc: 0.6293 - val_loss: 2.7940 - val_acc: 0.4131
Epoch 1968/2000

### Prediction Model

In [13]:
input2_ = tf.keras.layers.Input(shape=(1,))

x = embed_layer(input2_)
x,h,c = lstm_layer_0(x, initial_state = [initial_h, initial_c])
#x,h,c = lstm_layer_1(x)

output2_ = dense_layer(x)

pred_model = tf.keras.models.Model([input2_, initial_h, initial_c], [output2_,h,c])
pred_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           multiple             461800      input_4[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 64)]         0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 64)]         0                                            
____________________________________________________________________________________________

In [16]:
test_input_word = '<sos>'
test_input_word_idx = word2Idx[test_input_word]
test_input_word_vector = np.array([[test_input_word_idx]])

print (test_input_word_vector.shape)

h = np.zeros((1,LSTM_UNITS))
c = np.zeros((1,LSTM_UNITS))

output_poem = []
eos = word2Idx['<eos>']
idx2word = {v:k for k, v in word2Idx.items()}

for i in range(max_seq_len_from_data):
    
    print ('Start word is ',test_input_word_vector)
    o,h,c = pred_model.predict([test_input_word_vector, h, c])
    
    #print (o)
    probs = o[0,0]
    print ('found probs ',probs)
    if (np.argmax(probs)==0):
        print ('hell!')
    print ('found probs at index 0,',probs[0])
    probs[0] = 0
    probs = probs/probs.sum()
    print ('calculated probs ',probs)
    
    #idx = np.random.choice(len(probs), p=probs)
    idx = np.argmax(o[0,0])
    print (idx)
    if (idx == eos):
        break
    
    test_input_word_vector[0,0] = idx
    
    output_poem.append(idx2word.get(idx) + ' ')

(1, 1)
Start word is  [[15]]
found probs  [1.5133837e-15 1.2131329e-15 5.8814612e-13 ... 1.4160662e-11 1.6216838e-18
 6.6095421e-16]
found probs at index 0, 1.5133837e-15
calculated probs  [0.0000000e+00 1.2131329e-15 5.8814612e-13 ... 1.4160662e-11 1.6216838e-18
 6.6095421e-16]
16


In [15]:
print (output_poem)

[]
